# 一、运行此镜像


In [ ]:
#  直接在AutoDL中打开此镜像，依赖已经安装好，权重也下载了，还训练了两个个demo，一个叫langlang，一个叫nana
#  建议在终端运行代码。
#  建议在终端运行代码。
#  建议在终端运行代码。

# 二、准备自己的训练数据


In [ ]:
# 1、拍摄高清训练视频2-5分钟。
# 2、最好是绿幕背景，方便抠像，方便裁剪，方便后期拼接。
# 3、模特头部动作不能太大。
# 4、正常说话，随意说点什么就行，不要一直数数。
# 5、女生把头扎起来，男生不要有胡须。
# 6、用ffmpeg代码进行视频预处理，视频截取肩膀以上，大小512X512p，视频为25帧率，音频16000赫兹。
# 7、获取au.csv,并将视频和au.csv一同上传到data文件夹下的一个带名字的文件夹下。 如：data/nana/nana.mp4 
# 8、一定要记住训练视频从原视频裁剪下来得第一个像素点位置，方便后面进行视频拼接。

# 三、处理自己的数据集


In [ ]:
# 1、视频数据处理之一 
cd /root/LZZX-NeRF
python data_utils/process.py /root/LZZX-NeRF/data/nana/nana.mp4
# python data_utils/process.py /root/LZZX-NeRF/data/langlang/langlang.mp4

In [ ]:
# 2、视频数据处理之二 
# 眨眼数据处理
# Obtain AU45 for eyes blinking
# 获取眨眼 FeatureExtraction在OpenFace运行中，重新命名输出CSV文件并将其移动到data/langlang/au.csv
# 可以下载OpenFace并运行，生成.csv文件

In [ ]:
# 3、音频数据处理 : HuBERT
# # Borrowed from GeneFace. English pre-trained.
cd /root/LZZX-NeRF
python data_utils/hubert.py --wav data/nana/aud.wav
# python data_utils/hubert.py --wav data/langlang/aud.wav

# 四、训练自己的模型


In [ ]:
# 1、训练头部 train head ，建议训练总步数的一半。
cd /root/LZZX-NeRF
python main.py data/nana/ --workspace trial_nana/ -O --iters 100000 --asr_model hubert
# python main.py data/langlang/ --workspace trial_langlang/ -O --iters 100000 --asr_model hubert

In [ ]:
# 2、训练嘴型  lpips finetune
# cd /root/LZZX-NeRF
python main.py data/nana/ --workspace trial_nana/ -O --iters 125000 --finetune_lips --patch_size 32  --asr_model hubert
# python main.py data/langlang/ --workspace trial_langlang/ -O --iters 125000 --finetune_lips --patch_size 32  --asr_model hubert

In [ ]:
# # 训练身体 train (torso),根据需要进行训练,我建议不用训练。
# cd /root/LZZX-NeRF
# # python main.py data/nana/ --workspace trial_nana_torso/ -O --torso --head_ckpt trial_nana/checkpoints/ngp_ep0085.pth --iters 200000  --asr_model hubert
# # python main.py data/langlang/ --workspace trial_langlang_torso/ -O --torso --head_ckpt trial_langlang/checkpoints/ngp_ep0085.pth --iters 200000  --asr_model hubert

# 五、用自己训练的模型进行推理

In [ ]:
# # 1、先处理音频HUbert ,推理之前将音频转成16000赫兹，然后在用HUbert处理。
# ffmpeg -i /root/LZZX-NeRF/data/nana/demo000.mp3  /root/LZZX-NeRF/data/nana/demo000.wav
# python data_utils/hubert.py --wav data/nana/demo000.wav

In [ ]:
# 2、再加载训练后的权重进行推理，只驱动头部，身体直接拼接即可。
cd /root/LZZX-NeRF
python main.py  data/nana/ --workspace trial_nana/ -O --test --test_train  --aud  data/nana/demo000_hu.npy   --smooth_path  --asr_model hubert
# python main.py  data/langlang/ --workspace trial_langlang/ -O --test --test_train  --aud  data/langlang/test003_hu.npy   --smooth_path  --asr_model hubert
# # 源码作者的音视频合成

In [ ]:
# # 3、根据实际情况裁剪推理后的视频（如果分割得很好，就不需要执行这一步）。
# cd /root/LZZX-NeRF
# ffmpeg -i /root/generate-mp4/20231226075513.mp4   -vf  crop=512:400:0:0   /root/generate-mp4/temp.mp4

In [ ]:
# # 4、将推理后得视频拼接回原视频中去。（一定要记住视频得切割点，还要输入音频时长，否则拼接后对不齐）
# cd /root/LZZX-NeRF
# ffmpeg -ss 00:00:00    -i  /root/LZZX-NeRF/data/nana/280-300.mp4  -i  /root/generate-mp4/temp.mp4 -filter_complex "[0:v][1:v]overlay=280:300"  -t 25  -map 1:a -c:a copy  /root/generate-mp4/output.mp4

# 六、WEBUI推理


In [ ]:
# 启动webui界面，训练好模型后也可以直接启动界面进行推理
%cd /root/LZZX-NeRF
!python main-webui.py --autodl_url http://localhost:6006/
#!python main-webui.py --autodl_url http://u235859-a353-4b2d15c1.westb.seetacloud.com:8443/

/root/LZZX-NeRF
参数：Namespace(autodl_url='http://localhost:6006/', base_path='/root/LZZX-NeRF/')
nginx: [emerg] bind() to 0.0.0.0:1935 failed (98: Address already in use)
nginx: [emerg] bind() to 0.0.0.0:6006 failed (98: Address already in use)
webui访问地址：http://localhost:6006/
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
存储音频文件...
存储音频文件完成.
音频存储文件：/root/input/nana/17048551808289697.wav
音频预处理开始...
进行中...
音频处理cmd：python data_utils/hubert.py --wav /root/input/nana/17048551808289697.wav
SR: 24000 to 16000
(4055, 2, 1024)
音频预处理完成.
准备执行推理...
推理命令：python main.py ./data/nana --workspace trial_nana  -O  --test --test_train --aud /root/input/nana/17048551808289697_hu.npy --smooth_path --fps 25 --asr_model hubert 
进行中...
Namespace(H=450, O=True, W=450, amb_aud_loss=1, amb_dim=2, amb_eye_loss=1, asr=False, asr_model='hubert', asr_play=False, asr_save_feats=False, asr_wav='', att=2, aud='/root/input/nana/17048551808289697_hu.npy', bg_img='', 